In [87]:
import numpy as np

In [88]:
dataset = []
for filename in ["./dataset_raw/semeval2016-task6-trialdata.txt", "./dataset_raw/semeval2016-task6-trainingdata.txt"]:
    f = open(filename, 'r')
    dataset  = dataset + f.readlines()

print "Number of items = ", len(dataset)

Number of items =  2916


In [89]:
glove_word_vec_file = "/Users/bhargav/Downloads/glove.twitter.27B/glove.twitter.27B.25d.txt"

def readGloveData(glove_word_vec_file):
    f = open(glove_word_vec_file, 'r')
    rawData = f.readlines()
    word_vec_dict = {}
    for line in rawData:
        line = line.strip().split()
        tag = line[0]
        vec = line[1:]
        word_vec_dict[tag] = np.array(vec, dtype=float)
            
    return word_vec_dict
            
word_vec_dict = readGloveData(glove_word_vec_file)

In [90]:
def getWordVector(word, word_vec_dict):
    if word in word_vec_dict:
        return word_vec_dict[word]
    return np.zeros_like(word_vec_dict['hi'])
    

In [111]:
def readTweets(fileName):
    file = open(fileName, 'r')
    raw_tweets = []
    tweets = []
    tweetClass = []
    line = file.readline()
    for line in file:
        raw_tweets.append(line.lower().strip().replace(',',' ').split())
    for tweet in raw_tweets:       
        if tweet[1] != 'feminist':
            continue
            
        tweets.append(tweet[1:len(tweet) - 1])
        tweetClass.append(tweet[len(tweet) - 1])
    tweets.append(tweetClass)
    return tweets

def getTweetVectors(tweets):
    tweetVectors = []
    for tweet in tweets:
        #print getSumVectors(tweet, word_vec_dict)
        tweetVectors.append(getSumVectors(tweet, word_vec_dict))
    return tweetVectors

word_vec_dict = readGloveData('./glove.twitter.27B/glove.twitter.27B.25d.txt')
tweets = readTweets('./dataset_raw/semeval2016-task6-trainingdata.txt')



In [112]:
def getSumVectors(sentence, word_vec_dict):
    numNonZero = 0
    vector = np.zeros_like(word_vec_dict['hi'])
    for word in sentence:
        vec = getWordVector(word, word_vec_dict)
        #print word, vec
        
        vector = vector + vec
        if vec.sum() != 0:
            numNonZero += 1

    if numNonZero:
        vector = vector / numNonZero

    return vector

In [113]:
tweetVectors = getTweetVectors(tweets[0:len(tweets) - 1])
print tweets[0]
print getSumVectors(tweets[0], word_vec_dict)


['feminist', 'movement', 'always', 'a', 'delight', 'to', 'see', 'chest-drumming', 'alpha', 'males', 'hiss', 'and', 'scuttle', 'backwards', 'up', 'the', 'wall', 'when', 'a', 'feminist', 'enters', 'the', 'room.', '#manly', '#semst']
[ -2.37875286e-01   1.77393095e-01  -4.95440952e-02   2.19813429e-01
  -9.62951905e-02  -1.27029524e-01   8.26993810e-01  -3.21217724e-01
   2.68923571e-01  -2.55428905e-01  -2.05642381e-01   1.83934381e-01
  -3.77206286e+00  -4.64933333e-04  -8.93849952e-02   1.57279905e-01
   1.27070143e-01  -1.83224343e-01   1.02865857e-01  -8.50095238e-02
  -4.60011429e-02   2.93992857e-01   1.41085714e-01  -2.92647905e-01
  -1.36500286e-01]


In [114]:
tweetClasses = set(tweets[-1])

mapping = {'favor': 1, 'none': 0, 'against': 1}

tweetClasses = np.asarray([mapping[x] for x in tweets[-1]])
tweetData = np.asarray(tweetVectors)

print tweetClasses.shape
print tweetData.shape

(664,)
(664, 25)


In [115]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn import cross_validation

X = tweetData
Y = tweetClasses

print X.shape, Y.sum()

clf = SVC(kernel='rbf', gamma=1.5, random_state=34543)
scores = cross_validation.cross_val_score(clf, X, Y, cv=5)
print scores, scores.mean()


(664, 25) 538
[ 0.80597015  0.81203008  0.81203008  0.81060606  0.81060606] 0.810248484168
